In [ ]:
import sys
import os
from pathlib import Path
from sqlalchemy.engine import Engine
from sqlalchemy import create_engine, URL, text ,CursorResult
import plyvel
from typing import Any
%reload_ext sql

parent_path : Path = Path(sys.path[0]).parent.resolve()
if str(parent_path) not in sys.path:
    sys.path.append(str(parent_path))

from main.utils.runHealth import runHealth

# EDIT THIS LINE TO THE OPENWPM DATA WISH TO ANALYZE
datadir_path : Path = Path('/home/ndanner_plp/OpenWPM/crawl-data/datadir-0-10000')
database_url : URL = URL.create(drivername = "sqlite", database = str(datadir_path.joinpath("crawl-data.sqlite")) )
os.environ["DATABASE_URL"] = str(database_url)
engine : Engine = create_engine(database_url)
# if 'db' not in locals():
#     db : Any = plyvel.DB( str(datadir_path.joinpath("leveldb")) ) #type: ignore


In [ ]:
%%sql
SELECT *
FROM analysis_results
LIMIT 10

In [ ]:
%%sql
SELECT *
FROM sqlite_master

In [ ]:
%%sql
DROP TABLE analysis_results

In [ ]:
%%sql
SELECT COUNT(DISTINCT visit_id) 
FROM http_responses
WHERE response_status LIKE "2%"

In [ ]:
 
%%sql
SELECT response_status
FROM http_responses 
LIMIT 10

In [ ]:
n, s = runHealth(engine)
print(f"total visits: {n}, failed/incomplete visits: {s}. Success percentage: {round(100* (s/n)) }%")

In [ ]:
%%sql
SELECT visit_id, script_url, symbol, arguments, value, operation
FROM javascript
WHERE (visit_id,script_url) in (SELECT visit_id, url FROM http_responses WHERE content_hash = "0a63a423ac8fe788bf0660a1198d3b4288eef68f1d98892d4f86d151c3fe176d")

In [ ]:
%%sql
SELECT visit_id, url FROM http_responses WHERE content_hash = "0a63a423ac8fe788bf0660a1198d3b4288eef68f1d98892d4f86d151c3fe176d"

In [ ]:
%%sql
SELECT name FROM pragma_table_info('analysis_results')
WHERE name = "Canvas_Font_1M_Static"

In [ ]:
%%sql
SELECT DISTINCT value
FROM javascript
WHERE symbol = "HTMLCanvasElement.height" OR "HTMLCanvasElement.width"

In [ ]:
%%sql
SELECT COUNT(*)
FROM analysis_results
WHERE Canvas_1M_Static = 0 AND Canvas_1M_Dynamic = 1
LIMIT 10

In [ ]:
%%sql
SELECT COUNT(*) as n
FROM analysis_results

In [ ]:
# %%sql ["Cwm fjordbank glyphs vext quiz,"


with engine.connect() as conn:
    result: CursorResult[Any] = conn.execute(text("""
SELECT DISTINCT(value) 
FROM analysis_results AS ar
INNER JOIN javascript AS js
ON ar.visit_id = js.visit_id AND ar.script_url = js.script_url
WHERE ar.Canvas_Font_1M_Dynamic = True
AND js.symbol = "CanvasRenderingContext2D.font"
LIMIT 1000 
    """))
print( list(map(lambda row : row[0], result.all()) ) ) 


In [ ]:
%%sql
SELECT js.symbol, js.value, js.arguments, js.operation, js.visit_id, SUBSTRING(js.script_url,1, 32)
FROM analysis_results AS ar
INNER JOIN javascript AS js
ON ar.visit_id = js.visit_id AND ar.script_url = js.script_url
WHERE ar.WebGL_Dynamic = True
AND ( js.symbol LIKE "WebGLRenderingContext%" OR js.symbol = "CanvasRenderingContext2D.fillText" )
ORDER BY js.visit_id, js.script_url

In [ ]:
%%sql
WITH a AS(
SELECT COUNT(*)
FROM javascript AS js
WHERE (js.visit_id, js.script_url) IN (SELECT visit_id, script_url FROM analysis_results WHERE WebGL_Dynamic = True )
AND (js.visit_id, js.script_url) IN (SELECT DISTINCT visit_id, url  FROM http_responses WHERE content_hash <>"")
AND js.symbol = "CanvasRenderingContext2D.fillText" 
AND js.arguments LIKE '["Cwm fjordbank glyphs vext quiz%'
GROUP BY js.visit_id, js.script_url
)
SELECT COUNT(*)
FROM a

In [ ]:
%%sql 
WITH a AS(
SELECT COUNT(*)
FROM javascript AS js
WHERE (js.visit_id, js.script_url) IN (SELECT visit_id, script_url FROM analysis_results WHERE WebGL_Dynamic = True )
AND (js.visit_id, js.script_url) IN (SELECT DISTINCT visit_id, url  FROM http_responses WHERE content_hash <>"")
GROUP BY js.visit_id, js.script_url
)
SELECT COUNT(*)
FROM a

In [ ]:
%%sql 
WITH domain AS(
SELECT *
FROM javascript AS js
WHERE (js.visit_id, js.script_url) IN (SELECT visit_id, script_url FROM analysis_results WHERE WebGL_Dynamic = True )
AND (js.visit_id, js.script_url) IN (SELECT DISTINCT visit_id, url  FROM http_responses WHERE content_hash <>"")
),
total AS(
SELECT COUNT(*) AS total
FROM ( SELECT DISTINCT visit_id, script_url
    FROM domain
)
),
b AS(
SELECT COUNT(*) AS using_fillText_questionably
FROM ( SELECT * FROM 
    domain AS js
    WHERE js.symbol = "CanvasRenderingContext2D.fillText" 
    AND js.arguments LIKE '["Cwm fjordbank glyphs vext quiz%'
    GROUP BY js.visit_id, js.script_url
)
)
SELECT *
FROM total, b

In [ ]:
%%sql
WITH a AS(
    SELECT DISTINCT visit_id,url
    FROM http_responses
    WHERE content_hash <> ""
), b AS(
    SELECT DISTINCT visit_id,script_url
    FROM javascript
), c AS (
    SELECT COUNT(*) AS n
    FROM a
    INNER JOIN b
    ON a.visit_id = b.visit_id AND a.url = b.script_url
), a_count AS (
    SELECT COUNT(*) AS n
    FROM a
), b_count AS (
    SELECT COUNT(*) AS n
    FROM b
)
SELECT a_count.n, b_count.n, c.n
FROM a_count,b_count,c

In [ ]:
%%sql
WITH a AS(
    SELECT DISTINCT visit_id,url
    FROM http_responses
    WHERE content_hash <> ""
)
SELECT COUNT(*)
FROM a

In [ ]:
%%sql
SELECT COUNT(*)
FROM (
SELECT DISTINCT visit_id,url
FROM http_responses
WHERE content_hash <> ""
)

In [ ]:
%%sql
SELECT incomplete_visits.visit_id, site_visits.site_url
FROM incomplete_visits
INNER JOIN site_visits 
ON incomplete_visits.visit_id = site_visits.visit_id
LIMIT 10

In [ ]:
%%sql
SELECT symbol, count(DISTINCT visit_id) AS n_sites, count(*) as n
FROM javascript
GROUP BY symbol
ORDER BY symbol

In [ ]:
%%sql
SELECT COUNT(*)
FROM (
SELECT DISTINCT visit_id,url
FROM http_responses
WHERE content_hash <> ""
)


In [ ]:
%%sql
SELECT COUNT(*) FROM (
SELECT visit_id,url
FROM http_responses
WHERE content_hash <> ""
GROUP BY visit_id,url
)

In [ ]:
%%sql
WITH a AS (
SELECT visit_id,script_url
FROM javascript
GROUP BY visit_id,script_url
)
SELECT COUNT(*)
FROM a

In [ ]:
%%sql
SELECT value, COUNT(*) AS n
FROM javascript
WHERE symbol = "CanvasRenderingContext2D.font"
GROUP BY value
ORDER BY n DESC

In [ ]:
%%sql
SELECT *
FROM javascript
WHERE symbol = "window.navigator.webdriver"
LIMIT 10

In [ ]:
%%sql
SELECT COUNT(DISTINCT script_url) as n, arguments
FROM javascript
WHERE symbol = "window.matchMedia" AND arguments like "%null%"
GROUP BY arguments
ORDER BY n DESC
LIMIT 40

In [ ]:
%%sql
SELECT visit_id,script_url, COUNT(DISTINCT arguments) as n
FROM javascript
WHERE symbol = "window.matchMedia"
GROUP BY visit_id,script_url
ORDER BY n desc
LIMIT 40

In [ ]:
# %load_ext sql
# symbols = {}
# for path in Path("/home/ndanner_plp/OpenWPM/crawl-data").iterdir():
#     if path.is_dir() and re.match("^datadir" ,path.name):
#         %sql sqlite:///{path}/crawl-data.sqlite
#         result = %sql SELECT DISTINCT(symbol) FROM javascript
#         if result:
#             for symbol in result.DataFrame()['symbol']:
#                 if symbol in symbols:
#                     symbols[symbol].append(path.name)
#                 else:
#                     symbols[symbol] = [path.name]
# print( json.dumps( symbols, indent= 4 ) )

OfflineAudioContext

In [ ]:
%%sql
SELECT *
FROM javascript
LIMIT 10

In [ ]:
%%sql
WITH a AS (
SELECT visit_id,url, COUNT(*) as n
FROM http_responses
WHERE content_hash <> ""
GROUP BY visit_id,url
ORDER BY n DESC
)
SELECT a.visit_id, site_visits.site_url, a.url, a.n
FROM a
INNER JOIN site_visits
ON a.visit_id = site_visits.visit_id
LIMIT 20


In [ ]:
%%sql
WITH a AS (
SELECT visit_id,url, COUNT( DISTINCT content_hash ) as unique_content_hashs
FROM http_responses
WHERE content_hash <> ""
GROUP BY visit_id,url
ORDER BY unique_content_hashs DESC
)
SELECT a.visit_id, site_visits.site_url, a.url, a.unique_content_hashs
FROM a
INNER JOIN site_visits
ON a.visit_id = site_visits.visit_id
LIMIT 10


In [ ]:
%%sql
SELECT *
FROM http_responses
INNER JOIN site_visits
ON http_responses.visit_id = site_visits.visit_id
WHERE content_hash = "05240d0cdb821c6dedf4415fc448ae647edd3c3e286b43326c90c23756fa03e9" OR content_hash = "05e40ba8a6356921bf56db6cc53f34da4ca2e3ca5d63257dd965f0ff43705e93" OR content_hash = "072ed7ea062924ed3060069434fee5a33d0ad1910f1d8dc1d82d8ec4e6bb558a"

In [ ]:
# url LIKE "%cdn-cgi/challenge-platform/_/_/scripts/pica.js" 

In [ ]:
%%sql
WITH a AS(
SELECT *
FROM http_responses AS hr
WHERE EXISTS (
    SELECT *
    FROM analysis_results
    WHERE Canvas_Basic_Dynamic IS NULL AND Canvas_Basic_Static = 1 AND visit_id = hr.visit_id AND script_url = hr.url
)
AND url LIKE "%cdn-cgi/challenge-platform/_/_/scripts/pica.js"
GROUP BY visit_id,url
)
SELECT COUNT(*) 
FROM a


In [ ]:
%%sql    
    SELECT visit_id,script_url AS url
    FROM analysis_results
    WHERE Canvas_Basic_Dynamic IS NULL AND Canvas_Basic_Static = 1

In [ ]:
%%sql
WITH a AS(
SELECT visit_id, url AS script_url
FROM http_responses
WHERE content_hash = "2c06da86e7d6909e109262d832294260ba8aa5b121eafcb204e568a71493fd1b"
)
SELECT *
FROM javascript
WHERE (visit_id,script_url) IN a 
LIMIT 300

In [ ]:
import difflib
from pprint import pprint


x = difflib.Differ()
str1 = str( db.get(b"338359dbbf86b03f293159f6673b33f7f87a0ab0a6930df27bfc69582b824748"), encoding="utf-8")
str2 = str( db.get(b"339f38f18297db8afa9cdd0f873676e520ab736af47643d682ac158900f78913"), encoding="utf-8")
cmp = x.compare( str1.splitlines(keepends=True), str2.splitlines(keepends=True)  )
for line in filter(lambda s : s[0] == "+" or s[0] == "-", cmp ):
    print(line,end = "\n\n\n\n")


In [ ]:
%%sql
SELECT *
FROM site_visits
WHERE visit_id = "7805288562003358" OR visit_id = "6723308048261410"

In [ ]:
r = requests.get("http://localhost:8000", params={})

In [ ]:
r.text